In [2]:
import numpy as np 


In [3]:
%pip install matplotlib

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
%pip install qiskit

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: psutil>=5 in c:\users\sande\appdata\local\packages\pythonsoftwarefoundation.python.3.9_qbz5n2kfra8p0\localcache\local-packages\python39\site-packages (from qiskit-terra==0.20.1->qiskit) (5.9.0)



In [6]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute, transpile
from qiskit.tools.visualization import state_visualization
from qiskit.tools.visualization import plot_bloch_multivector
import random
import math
from scipy.optimize import minimize
import numpy as np  
import time 

In [7]:
# Trying to build a quantum circuit that gets the inner-product between two quantum states that encodes Tu's data

x = np.arange(0,8,1)
print(x)
y = [random.uniform(-1,1) for p in range(8)]    # can add a kind of randomvariate white-noise here
N = len(x)
print(y/np.linalg.norm(y))

[0 1 2 3 4 5 6 7]
[ 0.45705959 -0.29912752  0.38504192  0.32967921 -0.36994325  0.45068736
 -0.04848989  0.31991438]


In [8]:
   
no_of_qubits = np.ceil(np.log2(N))    # compute how many qubits needed to encode either x or y

norm_x = np.linalg.norm(x)    # Normalize the vectors so that the quantum state vector is always on the unit sphere 
norm_y = np.linalg.norm(y)
x = x/norm_x # normalise the data set x
y = y/norm_y  # normalise the data set y 
print(x,y)


[0.         0.08451543 0.16903085 0.25354628 0.3380617  0.42257713
 0.50709255 0.59160798] [ 0.45705959 -0.29912752  0.38504192  0.32967921 -0.36994325  0.45068736
 -0.04848989  0.31991438]


In [9]:
circ = QuantumCircuit(no_of_qubits + 1,1)
vec = np.concatenate((x,y))/np.sqrt(2)
vec


array([ 0.        ,  0.05976143,  0.11952286,  0.17928429,  0.23904572,
        0.29880715,  0.35856858,  0.41833001,  0.32318994, -0.2115151 ,
        0.27226575,  0.23311841, -0.26158938,  0.31868409, -0.03428753,
        0.22621363])

In [10]:
circ.initialize(vec, range(math.ceil(no_of_qubits) + 1))  # Initializing the quantum states using the atom coordinates of Tu
circ.h(math.ceil(no_of_qubits))                    # apply hadamard to bottom qubit
circ.measure(math.ceil(no_of_qubits),0)            # measure bottom qubit in computational basis
circ.draw()

»
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
c: 1/»
     »
«     ┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                                                                            ├»
«     │                                                                                                                                             │»
«q_1: ┤1                                                                                                                                            ├»
«     │  Initialize(0,0.059761,0.11952,0.17928,0.23905,0.29881,0.35857,0.41833,0.32319,-0.21152,0.27227,0.23312,-0.26159,0.31868,-0.034288,0.22621) │»
«q_2: ┤2                                                                                                                                            ├»
«     │                                                                                                                                             │»
«q_3: ┤3                                                                                                                                            ├»
«     └─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘»
«c: 1/═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════»
«                                                                                                                                                    »
«             
«q_0: ────────
«             
«q_1: ────────
«             
«q_2: ────────
«     ┌───┐┌─┐
«q_3: ┤ H ├┤M├
«     └───┘└╥┘
«c: 1/══════╩═
«           0

In [11]:
%pip install pylatexenc

Note: you may need to restart the kernel to use updated packages.


In [12]:

def inner_prod(vec1, vec2):
    #first check lengths are equal
    if len(vec1) != len(vec2):
        raise ValueError('Lengths of states are not equal')
        
    circ = QuantumCircuit(no_of_qubits+1,1)
    vec = np.concatenate((vec1,vec2))/np.sqrt(2)
    
    circ.initialize(vec, range(math.ceil(no_of_qubits)+1))
    circ.h(math.ceil(no_of_qubits))
    circ.measure(math.ceil(no_of_qubits),0)

    backend = Aer.get_backend('qasm_simulator')
    job = execute(circ, backend, shots=20000)

    result = job.result()
    outputstate = result.get_counts(circ)

    if ('0' in outputstate.keys()):
        m_sum = float(outputstate["0"])/20000
        print(m_sum)
        print(np.sqrt(2 - 2*(2*m_sum -1)))
    else:
        m_sum = 0
     
    return 2*m_sum-1
 

In [13]:
print("Classical Euclidean distance = " , np.sqrt((x-y)@(x-y)))
print("Quantum Euclidean Distance = " , np.sqrt(2-2*inner_prod(x, y)))
print("")
print(x,y)


Classical Euclidean distance =  1.13714240647541
0.6785
1.134019400186787
Quantum Euclidean Distance =  1.134019400186787

[0.         0.08451543 0.16903085 0.25354628 0.3380617  0.42257713
 0.50709255 0.59160798] [ 0.45705959 -0.29912752  0.38504192  0.32967921 -0.36994325  0.45068736
 -0.04848989  0.31991438]


In [14]:
#dimensions = 2
#dist1 = np.array([[inner_prod(mnvec, movec),inner_prod(mnvec, novec)],[inner_prod(mqvec, mnvec),inner_prod(movec, novec)]])
#dist2 = np.array([[inner_prod(novec, mqvec),inner_prod(movec, mqvec)], [inner_prod(mvec, nqvec), inner_prod(nvec, ovec)]])
#z1 = np.zeros((dimensions,dimensions))
#in_qmatrix = np.block([[z1,dist1],[dist2,z1]])
#print(in_qmatrix)


In [39]:
#dist1c = np.array([[z1vec,z2vec],[z3vec,z4vec]])
#dist2c = np.array([[z5vec, z6vec], [z7vec, z8vec]])
#in_cmatrix = np.block([[z1,dist1c],[dist2c, z1]])
#print(in_cmatrix)
#np.linalg.eigvalsh(in_cmatrix)

In [44]:
# Expanding the distance/bi-partite matrix in terms of Pauli matrices
#from qiskit.aqua import QuantumInstance, aqua_globals
#from qiskit.aqua.components.feature_maps import SecondOrderExpansion
#quantum_instance = QuantumInstance(backend=backend_sim,
 #                                  shots=20000,
  #                                 noise_model=noise_model,
   #                                coupling_map=coupling_map,
    #                               measurement_error_mitigation_cls=CompleteMeasFitter,
     #
     #                               cals_matrix_refresh_period=30)
#hamiltonian = MatrixOperator(in_qmatrix)
#hamiltonian_qubit_op = op_converter.to_weighted_pauli_operator(hamiltonian)

In [24]:
#num_qubits = 3
#circuit1 = QuantumCircuit(num_qubits)
#for i in range(num_qubits):
 # circuit1.x(i)


#circuit2 = QuantumCircuit(num_qubits)
#for i in range(num_qubits):
#  circuit2.id(i)
    
#swap_test_circuit = QuantumCircuit(2*num_qubits + 1,1)
#swap_test_circuit.compose(circuit1, qubits=[1,2,3], inplace=True )
#swap_test_circuit.compose(circuit2, qubits=[4,5,6], inplace=True )
#swap_test_circuit.h(0)
#for i in range(num_qubits):
 # swap_test_circuit.cswap(0,i+1,i+4)

#swap_test_circuit.h(0)
#swap_test_circuit.measure([0],[0])
#swap_test_circuit.draw()

In [46]:
%pip install qiskit-aqua


^C
Note: you may need to restart the kernel to use updated packages.


In [221]:
1 + 1.69 + 2.25 + (2.25*1.69)

8.7425

In [47]:
np.array([-0.3, -2.5])@np.array([-0.3, -2.5])

6.34

In [48]:
6.34/8.7425


0.7251930225907921

# SWAP TEST CIRCUIT IMPLEMENTATION AND NUMBERS

In [15]:
# First step is to encode the data into quantum states. 
#There are some techniques to do it, in this case Amplitude embedding was used.
from scipy.spatial import distance

A = np.array([3,6])    # I am not sure how big the vectors of Tu are, but should be easy to generalize. 
B = np.array([7,9])
A-B
c = np.linalg.norm(A-B)     # the Euclidean distance computed using numpy 

A_norm=np.linalg.norm(A)          
B_norm=np.linalg.norm(B)
Dist=distance.euclidean(A, B)
Z = round(A_norm**2 + B_norm**2)  # To normalize our quantum register, this is needed. 
 
# create phi and psi state with the data    
phi = np.array([A_norm/np.sqrt(Z),-B_norm/np.sqrt(Z)]) # The first quantum register (amplitude encoded) vector  
psi = []                            

for i in range(len(A)):
    psi.append(((A[i]/A_norm) /np.sqrt(2)))   # second quantum register initialized. 
    psi.append(((B[i]/B_norm) /np.sqrt(2)))   # so we concatenate both vectors to form a 1-d array of len(A) + len(B)
# Quantum Circuit
q1 = QuantumRegister(1,name='anc')          # initialize anciallary qubit
q2 = QuantumRegister(1,name='q2')           # I have written this code for only 1 qubit for the q2 register. we can opt for arbitrary lengths   
q3 = QuantumRegister(2,name ='q3')          # quantum register with 2 qubits because we encode psi which is a state of dimension 4
c = ClassicalRegister(1,name='c')           # one classical register for measurement purposes
qc= QuantumCircuit(q1,q2,q3,c)              

# states initialization
qc.initialize(phi, q2)
qc.initialize(psi, q3[0:2])
qc.barrier()                               # Totatl wavefunctions can be seen at the first barrier before the cswap test is complete

# The swap test protocol 
qc.h(q1[0])                                  
qc.cswap(q1[0], q2[0], q3[0])
qc.h(q1[0] )
qc.barrier()
qc.measure(q1,c)

qc.draw()


░ ┌───┐   ┌───┐ ░ ┌─┐
 anc: ─────────────────────────────────────────────────░─┤ H ├─■─┤ H ├─░─┤M├
              ┌──────────────────────────────┐         ░ └───┘ │ └───┘ ░ └╥┘
  q2: ────────┤ Initialize(0.50709,-0.86189) ├─────────░───────X───────░──╫─
      ┌───────┴──────────────────────────────┴───────┐ ░       │       ░  ║ 
q3_0: ┤0                                             ├─░───────X───────░──╫─
      │  Initialize(0.31623,0.43412,0.63246,0.55816) │ ░               ░  ║ 
q3_1: ┤1                                             ├─░───────────────░──╫─
      └──────────────────────────────────────────────┘ ░               ░  ║ 
 c: 1/════════════════════════════════════════════════════════════════════╩═
                                                                          0

In [19]:
from qiskit.visualization import plot_histogram
shots = 1000000
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=shots)
result = job.result()
countsqd = result.get_counts(qc)
x = abs((4*Z*(countsqd['0']/shots - 0.5)))
quant_Dist = round(np.sqrt(x),4)
decimals = 5
print('Euclidean Distance quantum: ', round(quant_Dist,decimals))    
print('Euclidean Distance classical: ', round(Dist,4))  




Euclidean Distance quantum:  4.9767
Euclidean Distance classical:  5.0
